In [1]:
!apt-get update 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null 
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz 
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [473 kB]
Hit:12 http://ppa.launchpad.net

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [3]:
!wget -O Employee_Compensation.csv https://data.sfgov.org/api/views/88g8-5mnd/rows.csv?accessType=DOWNLOAD

--2021-07-06 17:59:55--  https://data.sfgov.org/api/views/88g8-5mnd/rows.csv?accessType=DOWNLOAD
Resolving data.sfgov.org (data.sfgov.org)... 52.206.140.205, 52.206.140.199, 52.206.68.26
Connecting to data.sfgov.org (data.sfgov.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘Employee_Compensation.csv’

Employee_Compensati     [    <=>             ] 150.66M  6.74MB/s    in 22s     

2021-07-06 18:00:18 (6.83 MB/s) - ‘Employee_Compensation.csv’ saved [157980932]



In [4]:
df = spark.read.csv('Employee_Compensation.csv',header=True)
df.show(5)

+-----------------------+---------------+--------+---------+----+------------------+---------------+-------------------+----------+--------------------+-------------------+--------------------+-------------------+---------+--------+--------------+------------+----------+-----------------+--------------+--------------+------------------+
|Organization Group Code|Job Family Code|Job Code|Year Type|Year|Organization Group|Department Code|         Department|Union Code|               Union|         Job Family|                 Job|Employee Identifier| Salaries|Overtime|Other Salaries|Total Salary|Retirement|Health and Dental|Other Benefits|Total Benefits|Total Compensation|
+-----------------------+---------------+--------+---------+----+------------------+---------------+-------------------+----------+--------------------+-------------------+--------------------+-------------------+---------+--------+--------------+------------+----------+-----------------+--------------+--------------+---

In [5]:
from pyspark.sql.functions import col

df = df.withColumn("Salaries", col("Salaries").cast("int"))
df = df.withColumn("Overtime", col("Overtime").cast("int"))
df = df.withColumn("Other Salaries", col("Other Salaries").cast("int"))
df = df.withColumn("Total Salary", col("Total Salary").cast("int"))
df = df.withColumn("Retirement", col("Retirement").cast("int"))
df = df.withColumn("Health and Dental", col("Health and Dental").cast("int"))
df = df.withColumn("Other Benefits", col("Other Benefits").cast("int"))
df = df.withColumn("Total Benefits", col("Total Benefits").cast("int"))
df = df.withColumn("Total Compensation", col("Total Compensation").cast("int"))

In [6]:
df.schema

StructType(List(StructField(Organization Group Code,StringType,true),StructField(Job Family Code,StringType,true),StructField(Job Code,StringType,true),StructField(Year Type,StringType,true),StructField(Year,StringType,true),StructField(Organization Group,StringType,true),StructField(Department Code,StringType,true),StructField(Department,StringType,true),StructField(Union Code,StringType,true),StructField(Union,StringType,true),StructField(Job Family,StringType,true),StructField(Job,StringType,true),StructField(Employee Identifier,StringType,true),StructField(Salaries,IntegerType,true),StructField(Overtime,IntegerType,true),StructField(Other Salaries,IntegerType,true),StructField(Total Salary,IntegerType,true),StructField(Retirement,IntegerType,true),StructField(Health and Dental,IntegerType,true),StructField(Other Benefits,IntegerType,true),StructField(Total Benefits,IntegerType,true),StructField(Total Compensation,IntegerType,true)))

In [7]:
df.describe().show()

+-------+-----------------------+------------------+------------------+---------+------------------+--------------------+---------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|summary|Organization Group Code|   Job Family Code|          Job Code|Year Type|              Year|  Organization Group|Department Code|          Department|       Union Code|               Union|          Job Family|                 Job| Employee Identifier|         Salaries|          Overtime|   Other Salaries|     Total Salary|        Retirement| Health and Dental|    Other Benefits|    Total Benefits|Total Compensation|
+-------+-----------------------+------------------+------------------+---------+------------------+--------------------+---------------+-----

In [8]:
from pyspark.sql.functions import desc, lpad, col

grouped1 = (df.groupBy("Department").count().orderBy(desc("count")).filter(col("count") > 10000))

grouped1 = grouped1.withColumn('Department',lpad(col('Department'),20,' '))

grouped1.show()

+--------------------+-----+
|          Department|count|
+--------------------+-----+
|       Public Health|76675|
|   DPH Public Health|73093|
|Municipal Transporta|52260|
|MTA Municipal Transp|49144|
|HSA Human Services A|27586|
|          POL Police|26636|
|              Police|26066|
|      Human Services|25635|
|Public Utilities Com|20202|
|Recreation And Park |19179|
|REC Recreation & Par|18809|
|  Airport Commission|15850|
|AIR Airport Commissi|15686|
|     Fire Department|15670|
| FIR Fire Department|13731|
|Department Of Public|13555|
|DPW GSA - Public Wor|12849|
+--------------------+-----+



In [9]:
from pyspark.sql.functions import avg, desc

expressions = dict(Salaries="avg", Overtime="max", Retirement="max")
grouped2 = df.groupBy("Organization Group").agg(expressions)

grouped2.show()

+--------------------+-------------+-----------------+---------------+
|  Organization Group|max(Overtime)|    avg(Salaries)|max(Retirement)|
+--------------------+-------------+-----------------+---------------+
|    Community Health|       165675|71246.21467202606|          99306|
|   Public Protection|       321523| 94465.3469847123|         121952|
|Human Welfare & N...|        78722| 53690.1830765428|          68959|
|Culture & Recreation|       119429|36582.49528086218|          58859|
|Public Works, Tra...|       207395|69901.50642629132|          93399|
|General City Resp...|        40990|9782.984732824427|          20076|
|General Administr...|       130098|72171.27192218351|         128694|
+--------------------+-------------+-----------------+---------------+



In [10]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator


vectorAssembler = VectorAssembler(
    inputCols=[
        "Salaries",
        "Overtime",
        "Other Salaries",
        "Retirement",
        "Health and Dental",
        "Other Benefits",
    ],
    outputCol="features",
    handleInvalid="skip",
)
assembled = vectorAssembler.transform(df)
assembled = assembled.select(["features", "Total Compensation", "Organization Group"])

# Random test/train split.
train_df, test_df = assembled.randomSplit([0.7, 0.3])

# Define the model.
rf = RandomForestRegressor(
    numTrees=20,
    featuresCol="features",
    labelCol="Total Compensation",
)

# Train the model.
rf_model = rf.fit(train_df)

# Save the model.
rf_model.write().overwrite().save("rf_regression.model")

# Make predictions.
predictions = rf_model.transform(test_df)

# Evaluate the model.
r2 = RegressionEvaluator(
    labelCol="Total Compensation", predictionCol="prediction", metricName="r2"
).evaluate(predictions)
rmse = RegressionEvaluator(
    labelCol="Total Compensation", predictionCol="prediction", metricName="rmse"
).evaluate(predictions)
print("RMSE={} r2={}".format(rmse, r2))

RMSE=11847.435537960399 r2=0.9735921582653476


In [14]:
df = df.drop("Organization Group", "Department", "Department Code", "Union", "Year Type", "Job Family", "Job" )
df = df.withColumn("Organization Group Code", col("Organization Group Code").cast("int"))
df = df.withColumn("Job Family Code", col("Job Family Code").cast("int"))
df = df.withColumn("Job Code", col("Job Code").cast("int"))
df = df.withColumn("Year", col("Year").cast("int"))
df = df.withColumn("Union Code", col("Union Code").cast("int"))
df = df.withColumn("Employee Identifier", col("Employee Identifier").cast("int"))

In [15]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler

label_column = "Organization Group Code"
vectorAssembler = VectorAssembler(
    inputCols=df.columns,
    outputCol="features",
    handleInvalid="skip",
)
assembled = vectorAssembler.transform(df)

# Random test/train split.
train_df, test_df = assembled.randomSplit([0.7, 0.3])

# Define the model.
rf = RandomForestClassifier(
    numTrees=20,
    featuresCol="features",
    labelCol=label_column,
)

# Train the model.
rf_model = rf.fit(train_df)

# Make predictions.
predictions = rf_model.transform(test_df)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol=label_column, predictionCol="prediction", metricName="accuracy"
)
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))
results = predictions.select([label_column, "prediction"])

Test Error = 0.0653112
